In [1]:
import pyreadr, pickle, re, os
import numpy as np
import pandas as pd


import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
# N = 10 # How many genes to extract


# path = '/home/sam/Classes/Stats/Bayes/assignments/keyGeneExpressionBySubtype.RData'
# path = '/home/sam/scRNAseq/Xenium/Retina_expMatrix_clean.RData'
N = 300 # How many genes to extract

path = '/home/sam/scRNAseq/Xenium/Network_genes_Normalized.RData'
rdata = pyreadr.read_r(path) # WS
# rdata = pyreadr.read_r('/home/sam/Downloads/keyGeneExpressionBySubtype.RData') #Laptop
# genes = rdata['key_genes']

In [3]:
# Load data
df = rdata['Retina_expMatrix_candidateGenes']
df['Cluster'] = df['Cluster'].apply(lambda x: x if len(x.split('_')[0]) == 2 else '0' + x) # Standardize cluster names

# Load the list of indices for each network to use
class_net_genes = rdata['Class_indices'].to_numpy().ravel()
rgc_net_genes = rdata['RGC_indices'].to_numpy().ravel()
ac_net_genes = rdata['AC_indices'].to_numpy().ravel()
bc_net_genes = rdata['BC_indices'].to_numpy().ravel()

In [4]:

df['cluster'] = df['Cluster']

def encode_class(arr):
    '''This function will encode subtypes' cell classesbased on expert rules and is not intended for decoding'''
    custom_array = []

    for value in arr:
        if re.match(r'^\d{2}_', value):
            custom_array.append(0)
        elif value.startswith('AC_'):
            custom_array.append(1)
        elif value.endswith('Photoreceptors'):
            custom_array.append(2)
        elif value == '0MG (Mueller Glia)':
            custom_array.append(3)
        elif value.startswith('0BC'):
            custom_array.append(4)
        elif value.startswith('0RBC'):
            # Note this duplication is for simplicity of handling the 2 BC naming conventions
            custom_array.append(4)
        else:
            custom_array.append(5)
    return custom_array

# Function to generate a sort key for each subclass name
def sort_key(name):
    if name.startswith('0BC'):
        return 4
    elif name.startswith('0RBC'):
        return 4
    elif re.match(r'^\d{2}_', name):
        return 0
    elif name.startswith('AC_'):
        return 1
    elif name.endswith('Photoreceptors'):
        return 2
    elif name == '0MG (Mueller Glia)':
        return 3
    
    else:
        return 5

# Apply the sort_key function to each subclass name and sort the DataFrame
df['sort_key'] = df['Cluster'].apply(sort_key)
df.sort_values(by='sort_key', inplace=True)
df.drop(columns='sort_key', inplace=True)  # Optionally remove the sort key

class_arr = encode_class(df['Cluster'])

# Encode the categoric response 
le = LabelEncoder()
df['Cluster'] = le.fit_transform(df['Cluster'])

# Move the response to the end for simply manipulation
cluster_col = df.pop('Cluster')
dataset_col = df.pop('Dataset')
df.insert(len(df.columns), 'Cluster', cluster_col)
df.insert(len(df.columns), 'Class', class_arr)

display(df)

def create_mapping(df):
    # Extract the unique pairs of encoded cluster values and their corresponding class encodings
    unique_pairs = df[['Cluster', 'Class']].drop_duplicates()
    
    # Create a dictionary mapping from Cluster to Class
    mapping = dict(zip(unique_pairs['Cluster'], unique_pairs['Class']))
    
    return mapping

# Usage
mapping = create_mapping(df)

# Shuffle the data
df = shuffle(df, random_state=42)

# Split the data into input features and labels
X = df.iloc[:, :-3].values.astype(np.float32)
X = np.round(X*100)/100
y = df.iloc[:, -2].values.astype(np.compat.long)

# Convert data to PyTorch tensors
X = torch.from_numpy(X)
y = torch.from_numpy(y)

# Split the data into training and test sets
train_size = int(0.8 * len(df))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

,Kcnip4,Isl2,Glra1,Zic1,Syndig1l,Isl1,Pou3f1,Mmp9,Grm5,Cpne4,...,Glrb,Rbpms,Vamp1,Cspg4,Kcnq1ot1,Cdh5,Foxp1,cluster,Cluster,Class
rownames,,,,,,,,,,,,,,,,,,,,,
aRGC10_TTTGTCATCGCCTGAG.1,0.581697,0.92563,0.000000,0.0,0.000000,0.518247,0.0,1.347651,0.000000,0.870727,...,0.000000,1.224622,0.747942,0.0,0.000000,0.0,2.153310,04_FminiOFF,3,0
aRGC4_ACCGTAATCATTTGGG.1,0.417933,0.00000,0.000000,0.0,0.949320,0.000000,0.0,0.000000,0.917796,0.000000,...,0.496589,0.361727,0.502913,0.0,0.880912,0.0,0.000000,37_Novel,53,0
aRGC4_ACCGTAATCCTCTAGC.1,0.000000,0.00000,0.000000,0.0,0.501770,0.620176,0.0,0.000000,0.000000,0.697105,...,0.419215,0.643775,0.695561,0.0,0.782477,0.0,0.000000,08_Novel,7,0
aRGC4_ACCGTAATCGGCATCG.1,0.700281,0.00000,0.871270,0.0,0.474085,0.000000,0.0,0.000000,1.209090,0.000000,...,0.846013,0.616255,1.345877,0.0,1.000123,0.0,0.556282,26_Novel,42,0
aRGC4_ACCGTAATCTCCAACC.1,0.000000,0.00000,0.000000,0.0,1.048800,0.000000,0.0,0.000000,0.000000,0.000000,...,0.680279,0.925154,0.688942,0.0,1.027932,0.0,0.000000,12_ooDS_NT,28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bipolar5_ATACACTCACGA,0.000000,0.00000,1.043696,0.0,0.000000,0.000000,0.0,0.000000,0.876917,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0BC6,19,4
Bipolar5_ATGCTTTAGATG,0.000000,0.00000,0.000000,0.0,0.000000,1.047718,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0RBC (Rod Bipolar cell),24,4
Bipolar5_AGCACACCCCCT,0.000000,0.00000,0.000000,0.0,0.000000,0.464995,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0BC5A (Cone Bipolar cell 5A),15,4


In [5]:
class Net1(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    
class Net2(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc2 = nn.Linear(h1_size, h2_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    
class Net3(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc2 = nn.Linear(h1_size, h2_size)
        self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    
class Net4(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net4, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc2 = nn.Linear(h1_size, h2_size)
        self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    
class Net5(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net5, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc2 = nn.Linear(h1_size, h2_size)
        self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    
class Net6(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net6, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc2 = nn.Linear(h1_size, h2_size)
        self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)

In [17]:
# This cell is an experiment in training currciulumn
# The idea is to specifically train networks to learn one class only
# To do this we will train it head on against one other class ata a time
# When trainign is complete the network will have been exposed to the desired class at least 44 times
# It will see all non-desired classes at least once
# In this instantiation the network will not be weighted ahead of time to know what class is the desired class
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Step 1: Prepare your training data
def head2head(train_y, train_X, desired, noise):
    '''This function will take a desired category and a noise category and create a paired down dataset
    The paired down data set will be an equal mix of desired and noise
    If there are not enough noise examples, a random set will be duplicated to make the lengths match
    If there are too many noise samples, a random set will be excluded'''

    # Isolate the desired instances
    TS_desired_y = train_y[train_y == desired]
    TS_desired_X = train_X[train_y == desired]
    TS_noise_y = train_y[train_y == noise]
    TS_noise_X = train_X[train_y == noise]
    # Check if the desired class is longer, if so balance it with replicates from te noise
    if len(TS_desired_y) > len(TS_noise_y):
        delta = len(TS_desired_y) - len(TS_noise_y)
        indice = np.random.choice( range(len(TS_noise_y)), delta)
        indice = torch.tensor(indice)
        extra_y = TS_noise_y[indice]
        extra_X = TS_noise_X[indice]
        TS_noise_y = torch.cat((TS_noise_y,extra_y))
        TS_noise_X = torch.cat((TS_noise_X,extra_X))
    # Check if the desired class is shorter, if so randomely discard examples from the other
    elif len(TS_desired_y) < len(TS_noise_y):
        indice = np.random.choice( range(len(TS_noise_y)), len(TS_desired_y))
        indice = torch.tensor(indice)
        TS_noise_y = TS_noise_y[indice]
        TS_noise_X = TS_noise_X[indice]
    # Combine the two balanced data sets
    TS_y = torch.cat((TS_noise_y,TS_desired_y))
    TS_X = torch.cat((TS_noise_X,TS_desired_X))
    # Shuffle the data
    indice = np.random.choice( range(len(TS_y)), len(TS_y))
    indice = torch.tensor(indice)
    TS_y = TS_y[indice]
    TS_X = TS_X[indice]
    return TS_y, TS_X


def DesiredGeneCurriculum(train_y, train_X, desired=1):
    if desired!=None:
        # Create a training set using this function to create my desired class biased training set
        noise_inds = [i for i in np.unique(train_y) if i != desired]

    # Move the data to the GPU
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Convert the input data to the appropriate data type for the GPU
    train_X = train_X.to(device)
    train_y = train_y.to(device)

    ts_datasets = []
    if desired!=None:
        for noise_ind in noise_inds:
            TS_y, TS_X = head2head(train_y, train_X, desired=desired, noise=noise_ind)
            ts_datasets.append(torch.utils.data.TensorDataset(TS_X, TS_y))
    else:
        ts_datasets.append(torch.utils.data.TensorDataset(train_X, train_y))

    # Concatenate all datasets into one training set
    training_set = torch.utils.data.ConcatDataset(ts_datasets)
    
    return training_set

def QuickNN(training_set,n, num_epochs, model_dict, batch_size, mapping,
            target_subtype=None, class_weight=1,
            l1_lambda = 0.0, stopEarly = 10, visualize=False, returnLoss=False):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Step 2: Define your neural network
    # Instantiate the neural network model
    inputs = training_set[0][0].shape[0]
    model = model_dict['model'](input_size=inputs, num_classes=n,
            h1_size=n*model_dict['h1_size'], h2_size=n*model_dict['h2_size'], 
                            h3_size=n*model_dict['h3_size'], dropout_prob=model_dict['dropout_prob']
            ).to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_lambda, amsgrad=True)

    # Define the learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                          factor=0.05, patience=5)
    
    # Step 3: Set up your training loop
    train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    best_loss = float('inf')  # initialize the best validation loss
    early_stop_counter = 0  # initialize the early stopping counter

    if stopEarly > 0:
        print("Early Stopping Initialized")
        # Create the validation set
        val_size = int(len(training_set) * 0.2) # Use 20% of the training set for validation
        val_set, train_set = random_split(training_set, [val_size, len(training_set) - val_size])
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if len(inputs) == 0:
                continue
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        train_losses.append(epoch_loss / len(train_loader))

        if stopEarly > 0:
            
            # Evaluate the model on the validation set
            with torch.no_grad():
                val_loss = 0.0
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
            val_loss /= len(val_loader)
            val_losses.append(val_loss)

            # Check if the validation loss has improved
            if np.round(val_loss,5) < best_loss:
                best_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= stopEarly:  # if the validation loss hasn't improved for 10 epochs, stop training
                    print(f"Early stopping at: Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
                    break
        else:
            # Update the learning rate using the scheduler
            lr_scheduler.step(loss)


        if visualize:
            # Print the training loss and learning rate after every epoch
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
    if returnLoss:
        return model, (train_losses, val_losses)
    else:
        return model

def TestModel(test_X, test_y, model, le, visualize=True):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_X = test_X.to(device)
    test_y = test_y.to(device)

    # Evaluate the model on the test set
    with torch.no_grad():
        outputs = model(test_X)
        _, predicted = torch.max(outputs.data, 1)

    results = pd.DataFrame()
    for i in range(min(y), max(y)+1):
        cells = sum(test_y==i).item()
        test_y_i = test_y==i
        y_pred_i = predicted==i
        TP = sum((test_y_i==1) & (y_pred_i==1)).item()
        FP = sum((test_y_i==0) & (y_pred_i==1)).item()
        TN = sum((test_y_i==0) & (y_pred_i==0)).item()
        FN = sum((test_y_i==1) & (y_pred_i==0)).item()
        TPR = TP / np.where(TP+FN == 0, np.nan, TP+FN)
        TNR = TN / np.where(TN+FP == 0, np.nan, TN+FP)
        Prec = TP / np.where(TP+FP == 0, np.nan, TP+FP)
        F1 = 2 * (Prec * TPR) / np.where((Prec + TPR) == 0, np.nan, Prec + TPR)
        Accuracy = (TP+TN) / np.where(TP+FP+FN+TN == 0, np.nan, TP+FP+FN+TN)

        res_i = {'Cluster' : le.inverse_transform([i])[0],
            'cells' : cells,
            'TP' : TP,
            'FP' : FP,
            'TN' : TN,
            'FN' : FN,
            'TPR' : TPR,
            'TNR' : TNR,
            'Prec' : Prec,
            'F1' : F1,
            'Accuracy' : Accuracy}
        
        res_i = pd.DataFrame([res_i])
        results = pd.concat([results,res_i], ignore_index=True)
        
    if visualize:
        display(results.sort_values(by=['Cluster']))

    return results

def class_decoder(class_index):
    if class_index == 0:
        return 'RGC'  # or the appropriate class name
    elif class_index == 1:
        return 'AC'  # replace with the actual class name
    elif class_index == 2:
        return 'Ph'  # adjust as necessary
    elif class_index == 3:
        return 'MG'  # or the correct class name
    elif class_index == 4:
        return 'BC'  # or '0RBC', depending on your class naming convention
    # Add additional cases as needed
    else:
        return 'Other'

def TestModelClass(test_X, test_y, model, le, mapping, visualize=True):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_X = test_X.to(device)
    test_y = test_y.to(device)
    
    # Transform test_y from subclass labels to class labels
    test_y = torch.tensor([mapping[subclass_label.item()] for subclass_label in test_y], device=device)

    # Evaluate the model on the test set
    with torch.no_grad():
        outputs = model(test_X)
        # Convert logits to probabilities
        probabilities = torch.exp(outputs)
        # Initialize tensor for aggregated class probabilities
        class_probs = torch.zeros(probabilities.shape[0], len(set(mapping.values())), device=device)

        # Aggregate probabilities for each class
        for subclass, class_ in mapping.items():
            class_probs[:, class_] += probabilities[:, subclass]

    # Determine class predictions
    _, class_predictions = torch.max(class_probs, 1)
    
    results = pd.DataFrame()
    for i in range(0, 6):
        cells = sum(test_y==i).item()
        test_y_i = test_y==i
        y_pred_i = class_predictions==i
        TP = sum((test_y_i==1) & (y_pred_i==1)).item()
        FP = sum((test_y_i==0) & (y_pred_i==1)).item()
        TN = sum((test_y_i==0) & (y_pred_i==0)).item()
        FN = sum((test_y_i==1) & (y_pred_i==0)).item()
        TPR = TP / np.where(TP+FN == 0, np.nan, TP+FN)
        TNR = TN / np.where(TN+FP == 0, np.nan, TN+FP)
        Prec = TP / np.where(TP+FP == 0, np.nan, TP+FP)
        F1 = 2 * (Prec * TPR) / np.where((Prec + TPR) == 0, np.nan, Prec + TPR)
        Accuracy = (TP+TN) / np.where(TP+FP+FN+TN == 0, np.nan, TP+FP+FN+TN)

        res_i = {'Class' : class_decoder(i),
            'cells' : cells,
            'TP' : TP,
            'FP' : FP,
            'TN' : TN,
            'FN' : FN,
            'TPR' : TPR,
            'TNR' : TNR,
            'Prec' : Prec,
            'F1' : F1,
            'Accuracy' : Accuracy}
        
        res_i = pd.DataFrame([res_i])
        results = pd.concat([results,res_i], ignore_index=True)
        
    if visualize:
        display(results.sort_values(by=['Class']))
    return(results)


def PickleSalad(save_dir):
        # Initialize a dictionary to store all experiment results
    all_experiment_results = {}

    # Loop through each file in the directory
    for filename in os.listdir(save_dir):
        if filename.startswith("experiment_results_") and filename.endswith(".pkl"):
            file_path = os.path.join(save_dir, filename)
            # Load the experiment result
            with open(file_path, 'rb') as f:
                experiment_data = pickle.load(f)
            # Use filename as a unique key
            experiment_key = filename.replace('.pkl', '')
            all_experiment_results[experiment_key] = experiment_data

    # Save the consolidated results to a pickle file
    all_results_pickle_path = os.path.join(save_dir, "all_results/all_experiment_results.pkl")
    os.makedirs(os.path.dirname(all_results_pickle_path), exist_ok=True)
    with open(all_results_pickle_path, 'wb') as f:
        pickle.dump(all_experiment_results, f)

    # Load the consolidated experiment results
    with open(all_results_pickle_path, 'rb') as f:
        consolidated_results = pickle.load(f)

    # Initialize empty DataFrames to store the filtered results
    class_results_df = pd.DataFrame()
    subtype_results_df = pd.DataFrame()

    # Loop through the consolidated experiment results
    for key, data in consolidated_results.items():
        # Process class-level results
        test_class_results = pd.DataFrame(data['class_results'])
        test_class_results['replicate'] = data['seed']
        for param_key, param_value in data['hyperparameters'].items():
            test_class_results[param_key] = param_value
        class_results_df = pd.concat([class_results_df, test_class_results], ignore_index=True)

        # Process subclass-level results
        test_subtype_results = pd.DataFrame(data['results'])
        test_subtype_results['replicate'] = data['seed']
        for param_key, param_value in data['hyperparameters'].items():
            test_subtype_results[param_key] = param_value
        subtype_results_df = pd.concat([subtype_results_df, test_subtype_results], ignore_index=True)

    # Save the filtered class-level results to a CSV file
    csv_file_path_class = os.path.join(save_dir, 'experiment_results_perClass.csv')
    class_results_df.to_csv(csv_file_path_class, index=False)

    # Save the filtered subclass-level results to a CSV file
    csv_file_path_subtype = os.path.join(save_dir, 'experiment_results_perSubtype.csv')
    subtype_results_df.to_csv(csv_file_path_subtype, index=False)

    print("Filtered experiment results saved successfully.")

In [19]:
# # Convert train_X and test_X to PyTorch tensors on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_X = train_X.to(device)
test_X = test_X.to(device)


# Directory to save experimental data
save_dir = "/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment0/"

# Check if the directory exists, if not create it
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

n = len(np.unique(y))
num_epochs = 100 # specify the number of epochs to train for
batch_size = 32 # specify the batch size for training
# Define hyperparameters to test
num_epochs_options = [10, 25, 50, 100]
early_stopping_options = [0, 5, 10]
l1_lambda_options = [0.0, 0.001, 0.005, 0.01]
    
model_grid = [{"model" : Net1, 'h1_size' : 1, 'h2_size' : 0, 'h3_size' : 0, 'dropout_prob' : 0},
             {"model" : Net1, 'h1_size' : 2, 'h2_size' : 0, 'h3_size' : 0, 'dropout_prob' : 0},
             {"model" : Net1, 'h1_size' : 3, 'h2_size' : 0, 'h3_size' : 0, 'dropout_prob' : 0},

             {"model" : Net2, 'h1_size' : 1, 'h2_size' : 1, 'h3_size' : 0, 'dropout_prob' : 0},
             {"model" : Net2, 'h1_size' : 2, 'h2_size' : 2, 'h3_size' : 0, 'dropout_prob' : 0},
#              {"model" : Net2, 'h1_size' : 1, 'h2_size' : 2, 'h3_size' : 0, 'dropout_prob' : 0},
#              {"model" : Net2, 'h1_size' : 2, 'h2_size' : 1, 'h3_size' : 0, 'dropout_prob' : 0},
#              {"model" : Net2, 'h1_size' : 2, 'h2_size' : 3, 'h3_size' : 0, 'dropout_prob' : 0},
             {"model" : Net2, 'h1_size' : 3, 'h2_size' : 3, 'h3_size' : 0, 'dropout_prob' : 0},


             {"model" : Net3, 'h1_size' : 1, 'h2_size' : 1, 'h3_size' : 1, 'dropout_prob' : 0},
             {"model" : Net3, 'h1_size' : 2, 'h2_size' : 2, 'h3_size' : 2, 'dropout_prob' : 0},
             {"model" : Net3, 'h1_size' : 3, 'h2_size' : 3, 'h3_size' : 3, 'dropout_prob' : 0}]

# Initialize a dictionary to store results
experiment_results = {}

# Experiment loop
for seed in range(18,108,18):
    for model_dict in model_grid:
        l1, l2, l3 = n*model_dict['h1_size'], n*model_dict['h2_size'], n*model_dict['h3_size']
        for epochs in num_epochs_options:
            for early_stop in early_stopping_options:
                for l1_lambda in l1_lambda_options:
                    # Setting the seed for reproducibility
                    torch.manual_seed(seed)
                    np.random.seed(seed)
                    
                    # Create save path and check if it exists
                    experiment_id = f"epochs_{epochs}_earlyStop_{early_stop}_l1_{l1_lambda}_layers_{l1}_{l2}_{l3}_seed_{seed}"
                    model_path = os.path.join(save_dir, f"model_state_{experiment_id}.pt")
                    result_pickle_path = os.path.join(save_dir, f"experiment_results_{experiment_id}.pkl")
                    
                    # Check if the experiment has not been done
                    if not os.path.exists(result_pickle_path):
                        print(f'Testing seed: {seed}, layers: {l1}_{l2}_{l3} trained with {epochs} epochs, early stopping at {early_stop} and regularization of {l1_lambda}')


                        # Prepare the training dataset
                        training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)

                        # Instantiate and train the model
                        model, loss_history = QuickNN(training_set, n=n, 
                                                      num_epochs=epochs, model_dict=model_dict, 
                                        batch_size=batch_size, mapping=mapping, class_weight=0,
                                        l1_lambda=l1_lambda, stopEarly=early_stop, visualize=False, returnLoss=True)
                        model = model.to(device)

                        # Evaluate the model on the test set
                        results = TestModel(test_X, test_y, model, le, visualize=False)
                        class_results = TestModelClass(test_X, test_y, model, le, mapping, visualize=False)

                        # Store the results
                        experiment_results[experiment_id] = {
                            'seed' : seed,
                            'model_path': model_path,
                            'results': results,
                            'class_results' : class_results,
                            'hyperparameters': {
                                'num_epochs': epochs,
                                'early_stopping': early_stop,
                                'l1_lambda': l1_lambda,
                                'layer1' : l1,  
                                'layer2' : l2, 
                                'layer3' : l3
                            },
                            'loss_history': loss_history
                        }

                        # Instead of storing the model in experiment_results, save it directly to a file
                        torch.save(model.state_dict(), model_path)

                        # Save the experiment results after each iteration
                        with open(result_pickle_path, 'wb') as f:
                            pickle.dump(experiment_results[experiment_id], f)

                    print(f"Completed: {experiment_id}")
                    # Free up memory after each iteration
                    torch.cuda.empty_cache()

PickleSalad(save_dir)

Completed: epochs_10_earlyStop_0_l1_0.0_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_0_l1_0.001_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_0_l1_0.005_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_0_l1_0.01_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_5_l1_0.0_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_5_l1_0.001_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_5_l1_0.005_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_5_l1_0.01_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_10_l1_0.0_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_10_l1_0.001_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_10_l1_0.005_layers_125_0_0_seed_18
Completed: epochs_10_earlyStop_10_l1_0.01_layers_125_0_0_seed_18
Completed: epochs_25_earlyStop_0_l1_0.0_layers_125_0_0_seed_18
Completed: epochs_25_earlyStop_0_l1_0.001_layers_125_0_0_seed_18
Completed: epochs_25_earlyStop_0_l1_0.005_layers_125_0_0_seed_18
Completed: epochs_25_earlyStop_0

Completed: epochs_50_earlyStop_0_l1_0.0_layers_375_375_375_seed_90
Testing seed: 90, layers: 375_375_375 trained with 50 epochs, early stopping at 0 and regularization of 0.001
Completed: epochs_50_earlyStop_0_l1_0.001_layers_375_375_375_seed_90
Testing seed: 90, layers: 375_375_375 trained with 50 epochs, early stopping at 0 and regularization of 0.005
Completed: epochs_50_earlyStop_0_l1_0.005_layers_375_375_375_seed_90
Testing seed: 90, layers: 375_375_375 trained with 50 epochs, early stopping at 0 and regularization of 0.01
Completed: epochs_50_earlyStop_0_l1_0.01_layers_375_375_375_seed_90
Testing seed: 90, layers: 375_375_375 trained with 50 epochs, early stopping at 5 and regularization of 0.0
Early Stopping Initialized
Early stopping at: Epoch 46/50, Loss: 0.0000, Learning rate: 0.001000
Completed: epochs_50_earlyStop_5_l1_0.0_layers_375_375_375_seed_90
Testing seed: 90, layers: 375_375_375 trained with 50 epochs, early stopping at 5 and regularization of 0.001
Early Stopping I

In [ ]:
# A mistake was made during saving in the training, this fixes it so we recover some of the computation
import os
import shutil
import pickle

# Original directory containing the files
original_dir = "/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment5/"

# Target directory where the files will be copied with new names
target_dir = "/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment0/"

# Ensure the target directory exists
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Loop through each file in the original directory
for filename in os.listdir(original_dir):
    if filename.endswith(".pkl"):
        # Construct the full file path
        file_path = os.path.join(original_dir, filename)

        # Load the pickle file
        with open(file_path, 'rb') as file:
            data = pickle.load(file)

        # Extract the hyperparameters
        l1 = data['hyperparameters']['layer1']
        l2 = data['hyperparameters']['layer2']
        l3 = data['hyperparameters']['layer3']
        epochs = data['hyperparameters']['num_epochs']
        early_stop = data['hyperparameters']['early_stopping']
        l1_lambda = data['hyperparameters']['l1_lambda']
        seed = data['seed']

        # Reconstruct the correct filename
        new_filename = f"epochs_{epochs}_earlyStop_{early_stop}_l1_{l1_lambda}_layers_{l1}_{l2}_{l3}_seed_{seed}.pkl"

        # New file path in the target directory
        new_file_path = os.path.join(target_dir, new_filename)

        # Copy the file to the new location with the new name
        shutil.copy2(file_path, new_file_path)
        print(f"Copied and renamed: {filename} to {new_filename}")

